In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame as df

# train_data = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv')
# td_card = np.unique(train_data['card_id']) # 201917

# ratio = 0.005
# card_num = int(np.size(td_card, 0) * ratio)
# sampled_card = np.random.choice(td_card, card_num)
# del td_card

# samp_c = df(sampled_card,columns=['card_id'])
# del sampled_card

# filter1 = train_data["card_id"].isin(samp_c.card_id.tolist())
# td = train_data[filter1]
# del train_data, filter1

# print("Reading historical_transactions...............")
# historical_data = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv')
# filter2 = historical_data["card_id"].isin(samp_c.card_id.tolist())
# hd = historical_data[filter2]
# del historical_data, filter2

# print("Reading new_merchant_transactions...............")
# merchant_data = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv')
# filter3 = merchant_data["card_id"].isin(samp_c.card_id.tolist())
# md = merchant_data[filter3]
# del merchant_data, filter3

# frames = [hd, md]
# hmd = pd.concat(frames)
# print(hmd)

In [ ]:
# td.to_csv(r'/kaggle/working/td005_1.csv')
# hmd.to_csv(r'/kaggle/working/hmd005_1.csv')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame as df
import datetime
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

# train_data = pd.read_csv('../input/elo-dmbi-project/td005_1.csv')

import gc

hist = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv', parse_dates=['purchase_date'])
print(hist.shape)

new_merch = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv', parse_dates=['purchase_date'])
print(new_merch.shape)

frames = [hist, new_merch]
transaction_data = pd.concat(frames)
print(transaction_data.shape)
del hist, new_merch
gc.collect()


# transaction_data = pd.read_csv('../input/elo-dmbi-project/hmd005_1.csv',parse_dates=['purchase_date'])


#delete inconsistent data
indexNames = transaction_data[ transaction_data['city_id'] == -1].index
transaction_data.drop(indexNames , inplace=True)

indexNames = transaction_data[ transaction_data['category_2'].isna() ].index
transaction_data.drop(indexNames , inplace=True)

indexNames = transaction_data[ transaction_data['installments'] == -1].index
transaction_data.drop(indexNames , inplace=True)

indexNames = transaction_data[ transaction_data['installments'] == 999].index
transaction_data.drop(indexNames , inplace=True)

indexNames = transaction_data[ transaction_data['purchase_amount'] > 1].index
transaction_data.drop(indexNames , inplace=True)

# indexNames = transaction_data[ transaction_data['authorized_flag'] == "N"].index
# transaction_data.drop(indexNames , inplace=True)

# dropping ALL duplicte values 
transaction_data.drop_duplicates(subset =['card_id','merchant_id','purchase_date' ], keep = 'first', inplace = True) 

transaction_data['subsector_id_copy'] = transaction_data['subsector_id']


In [ ]:
transaction_data  = transaction_data.fillna(0)
print(transaction_data.isna().sum())

In [ ]:
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

transactions = binarize(transaction_data)

transactions = pd.get_dummies(transactions, columns=['category_2', 'category_3', 'subsector_id'])
# transactions = pd.get_dummies(transactions, columns=['category_2', 'category_3'])


transactions['month_diff'] = ((datetime.datetime.today() - transactions['purchase_date']).dt.days)//30
transactions['month_diff'] += transactions['month_lag']

transactions[:5]

del transaction_data


In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

transactions = reduce_mem_usage(transactions)

In [ ]:
merchants = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')
merchants["category_1"] = merchants["category_1"].map({'Y':1, 'N':0})
merchants["category_4"] = merchants["category_4"].map({'Y':1, 'N':0})
merchants = merchants.rename(columns={"category_1": "mer_category_1", "category_2": "mer_category_2","category_4": "mer_category_4", "city_id":"mer_city_id"})

print("merchants file read ")

merchant_columns = ["merchant_id", "mer_category_4", "mer_city_id", "merchant_group_id" ]

merchants =merchants[merchant_columns]


transactions = pd.merge(transactions, merchants, on='merchant_id', how='left')

transactions["same_city"]=np.where(transactions['city_id'] == transactions['mer_city_id'] , 1, 0)
del merchants

In [ ]:
# print(transactions.columns)
# print(merchants.columns)
transactions[:10]

In [ ]:
transactions.to_csv(r'/kaggle/working/transactions.csv')
gc.collect()

In [ ]:
def aggregate_transactions(data):
    
    data.loc[:, 'purchase_date'] = pd.DatetimeIndex(data['purchase_date']).\
                astype(np.int64) * 1e-9
     
    agg_func = {
    'authorized_flag': ['mean'],
    'category_1': ['mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    'merchant_group_id': ['nunique'],

    'mer_category_4': ['mean'],
    
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'same_city': ['mean'],
        
#       'subsector_id': ['nunique'],  
    'subsector_id_copy': ['nunique'],
    'subsector_id_1':['mean'],
    'subsector_id_2':['mean'],
    'subsector_id_3':['mean'],
    'subsector_id_4':['mean'],
   'subsector_id_5':['mean'],
    'subsector_id_7':['mean'],
    'subsector_id_8':['mean'],
    'subsector_id_9':['mean'],
    'subsector_id_10':['mean'],
    'subsector_id_11':['mean'],
    'subsector_id_12':['mean'],
    'subsector_id_13':['mean'],
    'subsector_id_14':['mean'],
    'subsector_id_15':['mean'],
    'subsector_id_16':['mean'],
    'subsector_id_17':['mean'],
    'subsector_id_18':['mean'],
    'subsector_id_19':['mean'],
    'subsector_id_20':['mean'],
    'subsector_id_21':['mean'],
    'subsector_id_22':['mean'],
    'subsector_id_23':['mean'],
    'subsector_id_24':['mean'],
    'subsector_id_25':['mean'],
    'subsector_id_26':['mean'],
    'subsector_id_27':['mean'],
    'subsector_id_29':['mean'],
    'subsector_id_30':['mean'],
    'subsector_id_31':['mean'],
    'subsector_id_32':['mean'],
    'subsector_id_33':['mean'],
    'subsector_id_34':['mean'],
    'subsector_id_35':['mean'],
    'subsector_id_36':['mean'],
    'subsector_id_37':['mean'],
    'subsector_id_38':['mean'],
    'subsector_id_39':['mean'],
    'subsector_id_40':['mean'],
    'subsector_id_41':['mean'],
    
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_date': [np.ptp, 'min', 'max'],
#     'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean', 'max', 'min', 'std'],
        
    }
    
    agg_transactions = data.groupby(['card_id']).agg(agg_func)
    
    agg_transactions.columns = ['_'.join(col).strip() for col in agg_transactions.columns.values]
    agg_transactions.reset_index(inplace=True)
    
    temp_df = (data.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_transactions = pd.merge(temp_df, agg_transactions, on='card_id', how='left')
    
    return agg_transactions


agg_transactions = aggregate_transactions(transactions)
# agg_transactions = reduce_mem_usage(agg_transactions)

agg_transactions[:5]

del transactions
gc.collect()
# # scaling

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


columns_to_scale = [c for c in agg_transactions.columns if c not in ['card_id', 'Unnamed: 0']]

agg_transactions[columns_to_scale ] = scaler.fit_transform(agg_transactions[columns_to_scale])

print("writing to csv.........")
agg_transactions.to_csv(r'/kaggle/working/agg_transactions.csv')


In [ ]:
agg_transactions.isna().sum()[:10]
agg_transactions[:5]
agg_transactions.to_csv(r'/kaggle/working/agg_transactions.csv')

In [ ]:
# new=pd.DataFrame()
# new.to_csv(r'/kaggle/working/transactions.csv')
agg_transactions.shape

In [ ]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

# train = read_data('../input/elo-dmbi-project/td005_1.csv')
train = read_data('../input/elo-merchant-category-recommendation/train.csv')
print("training file read ")

train = pd.merge(train, agg_transactions, on='card_id', how='left')
# train.drop(columns=['Unnamed: 0'])
train[['elapsed_time']] = scaler.fit_transform(train[['elapsed_time']])


indexNames = train[ abs(train['target']) > 10].index
train.drop(indexNames , inplace=True)

train = pd.get_dummies(train, columns=['feature_1', 'feature_2'])

train_split = np.split(train, [int(0.9*train.shape[0])] , axis=0)
train_data = train_split[0]
test_data = train_split[1]

In [ ]:
#corelation matrix

# f = plt.figure(figsize=(30, 30))
# plt.matshow(train.corr(), fignum=f.number)
# plt.xticks(range(train.shape[1]), train.columns, fontsize=14, rotation=45)
# plt.yticks(range(train.shape[1]), train.columns, fontsize=14)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=14)
# plt.title('Correlation Matrix', fontsize=16);


# print(train_data["target"].corr(train_data["purchase_date_min"]))

col_names = [c for c in train.columns if c not in ['card_id', 'first_active_month', 'Unnamed: 0','target']]

for colm in col_names:
    print(colm ," :  ", train["target"].corr(train[colm]) )

In [ ]:
features_sorted = data=best_features.sort_values(by="importance",
                                           ascending=False)
features_sorted = features_sorted["feature"].to_numpy()

for feat in features_sorted:
    print(feat ," :  ", train_data["target"].corr(train_data[feat]) )

In [ ]:
param = {'num_leaves': 105,
         'min_data_in_leaf': 66, 
         'objective':'regression',
         'max_depth': 8,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.733,
         "bagging_freq": 1,
         "bagging_fraction": 0.9597 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 2.251,
         "random_state": 133,
         "verbosity": -1}



#features
features = [c for c in train.columns if c not in ['card_id', 'first_active_month', 'Unnamed: 0','target']]

categorical_feats = ['feature_1','feature_2']

train_split = np.split(train, [int(0.7*train_data.shape[0])] , axis=0)

train_data = train_split[0]
validation_data = train_split[1]


print(train.shape[0])
print(train_data.shape[0])
print(validation_data.shape[0])


trn_data = lgb.Dataset(train_data[features],
                           label=train_data["target"],
                           categorical_feature=categorical_feats
                          )

val_data = lgb.Dataset(validation_data[features],
                           label=validation_data["target"],
                           categorical_feature=categorical_feats
                          )


num_round = 10000
    
clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                early_stopping_rounds = 100
               )

# print(features)

####plot feature importance

feature_importance_df = pd.DataFrame()

fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = features
fold_importance_df["importance"] = clf.feature_importance()
fold_importance_df["fold"] = 1
feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    

cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]


plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()

In [ ]:
param = {'num_leaves': 105,
         'min_data_in_leaf': 66, 
         'objective':'regression',
         'max_depth': 8,
         'learning_rate': 0.005,
         "boosting": "rf",
         "feature_fraction": 0.733,
         "bagging_freq": 1,
         "bagging_fraction": 0.9597 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 2.251,
         "random_state": 133,
         "verbosity": -1}



from sklearn.model_selection import KFold
folds = KFold(n_splits=5, shuffle=True, random_state=9)

oof = np.zeros(len(train_data))
predictions = np.zeros(len(test_data))

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_data.values, train_data["target"].values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_data.iloc[trn_idx][features],
                           label=train_data["target"].iloc[trn_idx],
                             )
    val_data = lgb.Dataset(train_data.iloc[val_idx][features],
                           label=train_data["target"].iloc[val_idx],
                            )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train_data.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_data[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, train_data["target"])**0.5))


In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
errors_new = clf.predict(test_data[features]) - test_data["target"] 

errors_new = errors_new/ np.std(errors_new)
plt.plot(errors_new ,'bo')

print('val RMSE:', mean_squared_error(test_data["target"],  clf.predict(test_data[features]), squared=False) )

In [ ]:
print(np.count_nonzero(abs(errors_new) > 2.5)/errors_new.shape[0])

In [ ]:
new_train_data  = train_data.fillna(0.0)

In [ ]:
new_train_data

In [ ]:
# linear regressin - ordinary least squares 
features = [c for c in train.columns if c not in ['card_id', 'first_active_month', 'Unnamed: 0','target']]

# train_split = np.split(train, [int(0.8*train.shape[0])] , axis=0)

# train_data = train_split[0]
# validation_data = train_split[1]

new_train_data  = train_data.fillna(0.0)
new_val_data  = test_data.fillna(0.0)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(new_train_data[features].to_numpy(), new_train_data["target"].to_numpy())

#predictions
predictions = model.predict(new_train_data[features].to_numpy())
# Calculate the absolute errors
errors = predictions - new_train_data["target"].to_numpy()

print("Rsq", model.score(new_train_data[features].to_numpy(), new_train_data["target"].to_numpy()))
print("co-efficients", model.coef_)

print('val RMSE:', mean_squared_error(new_train_data["target"], predictions, squared=False) )
print('val mean absolute error:', mean_absolute_error(new_train_data["target"], predictions) )
# print(predictions)


In [ ]:
from yellowbrick.regressor import cooks_distance
from yellowbrick.regressor import ResidualsPlot

cooks_distance(
    new_train_data[features].to_numpy(), new_train_data["target"].to_numpy(),
    draw_threshold=True,
    linefmt="C0-", markerfmt=","
)

visualizer = ResidualsPlot(model)

visualizer.fit(new_train_data[features], new_train_data["target"])  # Fit the training data to the visualizer
# visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the figure


In [ ]:
import matplotlib.pyplot as plt

plt.plot(errors ,'.')

In [ ]:
import statsmodels.api as sm

# mod_fit = sm.OLS(new_train_data["target"],new_train_data[features]).fit()
res = mod_fit.resid # residuals
fig = sm.qqplot(res,line='45')
plt.show()


In [ ]:
import statsmodels.api as sm

#Fit linear model to any dataset
model_2 = sm.OLS(new_train_data["target"],new_train_data[features])
results = model_2.fit()

#create instance of influence
influence = results.get_influence()

#leverage (hat values)
leverage = influence.hat_matrix_diag

#Cook's D values (and p-values) as tuple of arrays
cooks_d = influence.cooks_distance

#standardized residuals
standardized_residuals = influence.resid_studentized_internal

plt.plot(standardized_residuals ,'.')
plt.plot(cooks_d[0] ,'.')

# qq plot
res = mod_fit.resid # residuals
fig = sm.qqplot(res,line='45')
plt.show()


In [ ]:
print(np.count_nonzero(abs(standardized_residuals) > 2.5)/standardized_residuals.shape[0])

In [ ]:
#linear regression - random forrests
features = [c for c in train.columns if c not in ['card_id', 'first_active_month', 'Unnamed: 0','target']]

train_split = np.split(train, [int(0.8*train.shape[0])] , axis=0)

train_data = train_split[0]
validation_data = train_split[1]

new_train_data  = train_data.fillna(0.0)
new_val_data  = validation_data.fillna(0.0)

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data

rf.fit(new_train_data[features].to_numpy(), new_train_data["target"].to_numpy())

# Use the forest's predict method on the test data
predictions = rf.predict(new_val_data[features].to_numpy())
# Calculate the absolute errors
errors = predictions - new_val_data["target"].to_numpy()

# Print out the RMSE
print('val RMSE:', mean_squared_error(new_val_data["target"], predictions, squared=False) )
print('val mean absolute error:', mean_absolute_error(new_val_data["target"], predictions, squared=False) )


In [ ]:
#cooks distance - linear regression
from yellowbrick.regressor import cooks_distance
cooks_distance(
    new_train_data[features].to_numpy(), new_train_data["target"].to_numpy(),
    draw_threshold=True,
    linefmt="C0-", markerfmt=","
)

In [ ]:
#residual plots  - linear regression
import matplotlib.pyplot as plt

plt.plot(errors ,'bo')

In [ ]:
#XGBoost

from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators=10000, learning_rate=0.005, num_parallel_tree=2,max_depth=10)

# Add silent=True to avoid printing out updates with each cycle
my_model.fit(new_train_data[features].to_numpy(), new_train_data["target"].to_numpy(), verbose=False, early_stopping_rounds=500,  eval_set=[(new_val_data[features].to_numpy(), new_val_data['target'].to_numpy())])

predictions = my_model.predict(new_val_data[features].to_numpy())

print("RMSE : " + str(mean_squared_error(predictions, new_val_data["target"] )))

In [ ]:
predictions = my_model.predict(new_train_data[features].to_numpy())

In [ ]:
predictions

In [ ]:
results = pd.DataFrame(data=predictions,columns=['predictions'])
results["target"] = new_val_data['target']
results

In [ ]:
## feature importance - comparison with other models - not working fully  

def plot_feature_importances(clf, X_train, y_train=None, 
                             top_n=20, figsize=(20,20), print_table=False, title="Feature Importances"):
     
    __name__ = "plot_feature_importances"
    

    import matplotlib.pyplot as plt
    
    from xgboost.core     import XGBoostError
    from lightgbm.sklearn import LightGBMError
    
    try: 
        if not hasattr(clf, 'feature_importances_'):
            clf.fit(X_train.values, y_train.values.ravel())

            if not hasattr(clf, 'feature_importances_'):
                raise AttributeError("{} does not have feature_importances_ attribute".
                                    format(clf.__class__.__name__))
                
    except (XGBoostError, LightGBMError, ValueError):
        clf.fit(X_train.values, y_train.values.ravel())
            
    feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
    feat_imp['feature'] = X_train.columns
    feat_imp.sort_values(by='importance', ascending=False, inplace=True)
    feat_imp = feat_imp.iloc[:top_n]
    
    feat_imp.sort_values(by='importance', inplace=True)
    feat_imp = feat_imp.set_index('feature', drop=True)
    feat_imp.plot.barh(title=title, figsize=figsize)
    plt.xlabel('Feature Importance Score')
    plt.show()
    
    if print_table:
        from IPython.display import display
        print("Top {} features in descending order of importance".format(top_n))
        display(feat_imp.sort_values(by='importance', ascending=False))
        
    return feat_imp

X_train = train_data[features].fillna(0)
y_train = pd.DataFrame(train_data["target"]).fillna(0)

from xgboost              import XGBClassifier
from sklearn.ensemble     import ExtraTreesClassifier
from sklearn.tree         import ExtraTreeClassifier
from sklearn.tree         import DecisionTreeClassifier
from sklearn.ensemble     import GradientBoostingClassifier
from sklearn.ensemble     import BaggingClassifier
from sklearn.ensemble     import AdaBoostClassifier
from sklearn.ensemble     import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm             import LGBMClassifier


clfs = [ RandomForestClassifier()]
    
#     LogisticRegression(),
# #         LGBMClassifier(), 
#         BaggingClassifier(), GradientBoostingClassifier(), AdaBoostClassifier(),  RandomForestClassifier(), XGBClassifier(),]

for clf in clfs:
    try:
        _ = plot_feature_importances(clf, X_train, y_train, top_n=X_train.shape[1], title=clf.__class__.__name__)
    except AttributeError as e:
        print(e)



In [ ]:
#check what is happening here

def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
#___________________________________________________________
final_group =  aggregate_per_month(transactions) 
final_group[:10]

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import gc
import warnings
from bayes_opt import BayesianOptimization

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from datetime import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        

def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_csv("../input/elo-dmbi-project/td005_1.csv", index_col=0)
train = reduce_mem_usage(train)

target = train['target']
del train['target']

features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
#features = [f for f in features if f not in unimportant_features]
categorical_feats = [c for c in features if 'feature_' in c]

In [ ]:
def LGB_CV(
          max_depth,
          num_leaves,
          min_data_in_leaf,
          feature_fraction,
          bagging_fraction,
          lambda_l1
         ):
    
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(train.shape[0])

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical_feats)
    
        param = {
            'num_leaves': int(num_leaves),
            'min_data_in_leaf': int(min_data_in_leaf), 
            'objective':'regression',
            'max_depth': int(max_depth),
            'learning_rate': 0.01,
            "boosting": "gbdt",
            "feature_fraction": feature_fraction,
            "bagging_freq": 1,
            "bagging_fraction": bagging_fraction ,
            "bagging_seed": 11,
            "metric": 'rmse',
            "lambda_l1": lambda_l1,
            "verbosity": -1
        }
    
        clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 200)
        
        oof[val_idx] = clf.predict(train.iloc[val_idx][features],
                                   num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        gc.collect()
        
    return -mean_squared_error(oof, target)**0.5

In [ ]:
LGB_BO = BayesianOptimization(LGB_CV, {
    'max_depth': (4, 10),
    'num_leaves': (5, 130),
    'min_data_in_leaf': (10, 150),
    'feature_fraction': (0.7, 1.0),
    'bagging_fraction': (0.7, 1.0),
    'lambda_l1': (0, 6)
    })

In [ ]:
print('-'*126)

start_time = timer(None)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=20, acq='ei', xi=0.0)
timer(start_time)

In [ ]:
|   iter    |  target   | baggin... | featur... | lambda_l1 | max_depth | min_da... | num_le... |
22       | -4.349    |  0.9597   |  0.7335   |  2.251    |  7.133    |  66.01    |  105.2 